In [1]:
import numpy as np
import torch as th

import time

In [2]:
print("PyTorch version: ", th.__version__ )
print("CUDA available: ", th.cuda.is_available())
print("CUDA version: ", th.version.cuda)

PyTorch version:  0.4.0
CUDA available:  True
CUDA version:  9.1.85


In [3]:
dtype = th.float64
device = th.device("cpu")

In [4]:
# Utility functions

# return the lower triangle of A in column order i.e. vech(A)
def vech(A):
    count = 0
    c = A.shape[0]
    v = th.zeros(c * (c + 1) // 2,)
    for j in range(c):
        for i in range(j,c):
            v[count] = A[i,j]
            count += 1
    return th.tensor(v , device=device, dtype=dtype)

# vech2L   create lower triangular matrix L from vechA
def vech2L(v,n):
    count = 0
    L = th.zeros((n,n))
    for j in range(n):
        for i in range(j,n):
            L[i,j]=v[count]
            count += 1
    return th.tensor(L , device=device, dtype=dtype)

In [69]:
n = 3;
vechLk = th.tensor([  1.00000039208682, 
              0.02548044275764261, 
              0.3525161612610669,
              1.6669144815242515,
              0.9630555318946559,
              1.8382882034659822 ], device=device, dtype=dtype, requires_grad=True);

vechLl = th.tensor([  1.3353550436464964,
               0.9153272033682132,
               0.7958636766525028,
               1.8326931436447955,
               0.3450426931160630,
               1.8711839323167831 ], device=device, dtype=dtype, requires_grad=True);
Sym = th.tensor([[0,0,1],
                [0,1,0],
                [1,0,0]], device=device, dtype=dtype, requires_grad=False);
#Sym = th.tensor([[1,0,0],
#                [0,1,0],
#                [0,0,1]], device=device, dtype=dtype);

In [70]:
Lk = vech2L(vechLk,n);
Ll = vech2L(vechLl,n);

# apply symmetry projection on Ll

# th.t() is shorthand for th.transpose(X, 0,1)
PLl = th.t(Sym) @ Ll;

# build Ak, Al, Akl, invAkl, invAk, invAl

Ak = Lk@th.t(Lk);
Al = PLl@th.t(PLl);
Akl = Ak+Al

skl = 2**(3*n/2) * th.sqrt( th.pow(th.abs(th.det(Lk))*th.abs(th.det(Ll))/th.det(Akl) ,3) )
print(skl)

tensor(0.5334, dtype=torch.float64)


In [71]:
th.autograd.grad(skl, vechLk, retain_graph=True)

(tensor([ 0.4898,  0.0786, -0.0560,  0.1179, -0.1113, -0.1632], dtype=torch.float64),)

In [72]:
th.autograd.grad(skl, vechLl)

(tensor([ 0.3198, -0.0666, -0.1495, -0.0751, -0.0352, -0.1917], dtype=torch.float64),)